In [1]:
import numpy as np
import xarray as xr
import pandas as pd
# from google.colab import drive
import datetime
import glob

#### Get list of all files for desired year

In [12]:
yr_sel = 2021
fdir = '/mnt/tcnas01/musgrave/ryan_ridata/oper_stext/{yr_sel}/'.format(yr_sel=yr_sel)
#fname_test = '20072600AL9220_ships.txt'
all_files = glob.glob(fdir+'*.txt')
no_files = len(all_files)
fname_test = all_files[1]

#### Read in each file one at a time

Relevant information:
* TIME
* LAND (KM)
* Prob of RI for 25 kt RI threshold
* Prob of RI for 30 kt RI threshold
* Prob of RI for 35 kt RI threshold
* LAT (DEG N)
* LONG (DEG W)
* Name
* Case No.
* DATE
* TIME

#### Create ultimate dataframe

In [13]:
#test_lines
# case = test_lines[3].split()[2]
# date = test_lines[3].split()[3]
# name = test_lines[3].split()[1]
#start_time = test_lines[3].split()[4]
#for line in test_lines:
 #   if line.startswith('LAND (KM)'):
  #      print(line.split()[2:])

In [16]:
fname_test

'/mnt/tcnas01/musgrave/ryan_ridata/oper_stext/2021/21021712AL8521_ships.txt'

In [14]:
with open(fname_test) as fn:
    for line in fn:
        if line.startswith("TIME (HR)"):
            print(line)

TIME (HR)          0     6    12    18    24    36    48    60    72    84    96   108   120   132   144   156   168



In [15]:
open(fname_test).readlines()[2].split()[2]

'SAT'

In [6]:
RI_prob_df_ALL = pd.DataFrame()
##
#no_files = 
#for ifile in np.arange(0,no_files-1):
ifile = 48
fname_test = all_files[ifile]
file_test = open(fname_test)
test_lines = file_test.readlines()
#test_lines[1].split()
#basin_from_filename = fname_test[58:60]

##### Info we need
* ATCF ID: line after IR SAT
* DATE: line after IR SAT
* INIT TIME: line after IR SAT
* TIME: line starts with 'TIME
* STORM TYPE: line starts with Storm
* LAND: line starts with LAND
* LAT: line starts with LAT
* LON: line starts with LON
* VMAX: line starts with 'V (kt) NO LAND
* SHIPS-RII probs: line starts with SHIPS-RII
* SHIPS consensus probs: line starts with CONSENSUS

In [10]:
df_forecast = pd.DataFrame(columns=['TIME','Storm Type','DTL','LAT','LON','ATCF ID','DATE','Init Time','VMAX',
                                   'RI Type','Pr(30/24)'])
RI_type = '   SHIPS-RII'
for x in np.arange(0,len(all_files)):
    x_file = open(all_files[x]).readlines()
    print('x file has been opened',all_files[x])
    df_fore_ALL = pd.DataFrame()
    for i in np.arange(0,len(x_file)):

        #i = 88
        iline = test_lines[i]
        #print(i)
        if iline.startswith('                                 *                  GFS version                   *\n'):
            continue
        if iline.startswith('                                 * IR SAT'):
            atcf_id = test_lines[i+1].split()[2]
            date = test_lines[i+1].split()[3]
            init_time = test_lines[i+1].split()[4]
            print(test_lines[i+1].split())
        if iline.startswith('TIME (HR)'):
            time = iline
            vmax = test_lines[i+1].split()[4]
            #print(time)
        #else:
         #   continue
        #
        df_forecast['TIME'] = time.split()[2:]
        ## 
        #
        if iline.startswith('Storm Type'):
            storm_type = iline
        df_forecast['Storm Type'] = storm_type.split()[2:]
        if iline.startswith('LAND'):
            DTL = iline
        df_forecast['DTL'] = DTL.split()[2:]
        if iline.startswith('LAT'):
            lat = iline
        if iline.startswith('LON'):
            lon = iline
        #
        df_forecast['LAT'] = lat.split()[3:]
        df_forecast['LON'] = lon.split()[2:]
        df_forecast['ATCF ID'] = atcf_id
        df_forecast['DATE'] = date
        df_forecast['Init Time'] = init_time

        df_forecast['VMAX'] = vmax
        ## 
        # df_fore_copy = df_forecast.copy()
        ##
        if test_lines[i].startswith('Matrix'):
           # xxx = test_lines[iline+3]
            xxx = test_lines[i+4].split()[3][0:4]
           # print(RI_30_24)
            #RI_type = RI_type
            # print(RI_type)
        #print(xxx)
        #print(RI_30_24)
        #print(xxx)
        df_forecast['RI Type'] = 'SHIPS-RII'
        df_forecast['Pr(30/24)'] = xxx
    df_fore_ALL = df_fore_ALL.append(df_forecast)

x file has been opened /mnt/tcnas01/musgrave/ryan_ridata/oper_stext/2021/21020212AL8321_ships.txt


NameError: name 'time' is not defined

In [11]:
iline

'                                 * ATLANTIC     2020 SHIPS INTENSITY FORECAST     *\n'

In [ ]:
test_lines[88].startswith('Matrix')
#np.array(test_lines[92].split()[3][0:4])
#df_forecast
#rint(test_lines[88].split())#[3][0:4])

In [ ]:
if yr_sel == 2021:
    line_no_vec = [1,3,90,92,95]
elif yr_sel == 2020:
    line_no_vec = [1,3,90,91]#,95]
# Check formatting

# Check if extratropical
if np.isin('EXTP',test_lines[9].split()[0:7]):
    print('Extratropical')
    continue
#else:
 #   print('We good')
# Check if stays away from land
if test_lines[25].split()[0] == 'LAND':
    DTL_str = test_lines[25].split()[2:7]
    lat_str = test_lines[26].split()[3:8]
    lon_str = test_lines[27].split()[3:8]
    if np.isin('N/A',DTL_str):
        print('too close to land')
        continue
    else:
        DTL = [int(i) for i in DTL_str]
        #fname_test
        if all(x > 100 for x in DTL):
            print('sufficiently far from land')
        else:
            print('too close to land')
            continue
elif test_lines[25].split()[0] == 'LAT':
    DTL_str = test_lines[24].split()[2:7]
    lat_str = test_lines[25].split()[3:8]
    lon_str = test_lines[26].split()[3:8]
    if np.isin('N/A',DTL_str):
        print('too close to land')
        continue
    else:
        DTL = [int(i) for i in DTL_str]
        #fname_test
        if all(x > 100 for x in DTL):
            print('sufficiently far from land')
        else:
            print('too close to land')
            continue
elif test_lines[25].split()[0] == 'LONG':
    DTL_str = test_lines[23].split()[2:7]
    lat_str = test_lines[24].split()[3:8]
    lon_str = test_lines[25].split()[3:8]
    if np.isin('N/A',DTL_str):
        print('too close to land')
        continue
    else:
        DTL = [int(i) for i in DTL_str]
        #fname_test
        if all(x > 100 for x in DTL):
            print('sufficiently far from land')
        else:
            print('too close to land')
            continue
#print(basin_from_filename)

if test_lines[0].split()[1]  != 'GFS':
    line_no_use = [x - 1 for x in line_no_vec]
    print('formatting change :-/')
else:
    line_no_use = line_no_vec
    #print('normal format')
# Line 1 of each file contains information about BASIN, YEAR
if basin_from_filename == 'AL':
    basin = test_lines[line_no_use[0]].split()[1]
else:
    basin = test_lines[line_no_use[0]].split()[1:3]
## 
year = test_lines[line_no_use[0]].split()[2]
atcf_id = test_lines[3].split()[2]
# Line 3 of each file contains information about case, date, time
case = test_lines[line_no_use[1]].split()[2]
name = test_lines[line_no_use[1]].split()[1]
if len(test_lines[line_no_use[1]].split()) == 7:
    date = test_lines[line_no_use[1]].split()[3]
else:
    date = test_lines[line_no_use[1]+1].split()[3]
time = test_lines[line_no_use[1]].split()[4]

# RI probability headers are in lines 90
if len(test_lines[line_no_use[2]].split()) < 18:
    line_no_use[2] = line_no_use[2] - 1
RI_prob_header = test_lines[line_no_use[2]].split()
if test_lines[line_no_use[3]].split()[0] == 'Logistic:':
    SHIPS_RII = test_lines[line_no_use[3]-1].split()
elif test_lines[line_no_use[3]].split()[0] != 'SHIPS-RII:':
    SHIPS_RII = test_lines[line_no_use[3]+1].split()
else:
    SHIPS_RII = test_lines[line_no_use[3]].split()
######
if yr_sel != 2021:
    if test_lines[line_no_use[4]].split()[0] == 'DTOPS:':
        SHIPS_consensus = test_lines[line_no_use[4]-1].split()
    else:
        SHIPS_consensus = test_lines[line_no_use[4]-1].split()
else:
    SHIPS_consensus = np.empty(9)
    SHIPS_consensus[:] = np.nan

# Make dataframe
RI_prob_df = pd.DataFrame(columns={RI_prob_header[5],RI_prob_header[7],
                                  RI_prob_header[9],RI_prob_header[11],RI_prob_header[13],RI_prob_header[15],
                                  RI_prob_header[17],RI_prob_header[18]},index={'RII','Consensus'})
RI_prob_df = RI_prob_df.rename(columns={'|65/72':'65/72'})
RI_probs_list = RI_prob_df.columns
RI_prob_df = RI_prob_df.reindex(columns=sorted(RI_prob_df.columns))
RI_prob_df.loc['RII'] = SHIPS_RII[1::]
RI_prob_df.loc['Consensus'] = SHIPS_consensus[1::]

# Add case info
if basin_from_filename == 'EP':
    RI_prob_df['BASIN'] = basin[0]+'_'+basin[1]
elif basin_from_filename == 'CP':
    RI_prob_df['BASIN'] = 'CENTRAL_PACIFIC'
else:
    RI_prob_df['BASIN'] = basin
RI_prob_df['Date'] = pd.to_datetime(date)
RI_prob_df['ATCF ID'] = atcf_id
RI_prob_df['NAME'] = name
RI_prob_df['Time (hr)'] = time
RI_prob_df['CASE'] = case
RI_prob_df['LAT_0'] = np.float(lat_str[0]) if lat_str[0] != 'xxx.x' else np.nan
RI_prob_df['LON_0'] = np.float(lon_str[0]) if lon_str[0] != 'xxx.x' else np.nan
RI_prob_df_ALL = RI_prob_df_ALL.append(RI_prob_df)

In [ ]:
test_lines#[70].split()
test_lines#[line_no_use[4]].split()

In [ ]:
foo = np.float(lon_str[0]) if lon_str[0] != 'xxx.x' else np.nan
foo

In [ ]:
RI_prob_df_save = RI_prob_df_ALL.reset_index().rename(columns={'index':'RI Type'})
for icol in RI_probs_list:
    RI_prob_df_save[icol] = RI_prob_df_save[icol].str[:-1].astype(float)

In [ ]:
RI_prob_df_save

In [ ]:
RI_prob_df_save['Time (hr)'] = RI_prob_df_save['Time (hr)'].astype(int)
RI_prob_df_save['Time (TD)'] = pd.to_timedelta(RI_prob_df_save['Time (hr)'],unit='h')
#RI_prob_df_save_tr = RI_prob_df_save.where(RI_prob_df_save['20/12']<999.0).dropna(how='all')
#len(RI_prob_df_save_tr)
RI_prob_df_save['Time (TD)']

In [ ]:
#import datetime as dt
RI_prob_df_save['DATE_full'] = pd.to_datetime(RI_prob_df_save['Date']+RI_prob_df_save['Time (TD)']
                     ,format='%Y-%m-%d%H')
#RI_prob_df_save['Time (hr)']
#RI_prob_df_save['Date']
RI_prob_df_save['DATE_full']

In [ ]:
#
RI_prob_df_save.groupby(['BASIN','RI Type']).count()#.xs(('ATLANTIC','2020-09-14 12:00:00'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig1,ax1 = plt.subplots(1,1,figsize=(10,6))
sns.boxplot(data=RI_prob_df_save,x='BASIN',y='30/24',hue='RI Type')#,s=10,jitter=True,ax=ax1)
ax1.set_ylim([0,125])
ax1.grid()

In [ ]:
fname_save = 'DATA/SHIPS_RII_rapid_intense_probs_{yr}.csv'.format(yr=yr_sel)
RI_prob_df_save.to_csv(fname_save)